In [1]:
import gym
import ptan
import random
import torch
import torch.optim as optim
from ignite.engine import Engine

from lib import dqn_model, common

NAME = "02_n_steps"
DEFAULT_N_STEPS = 4

In [2]:
random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("PongNoFrameskip-v4")
env = ptan.common.wrappers.wrap_dqn(env)
env.seed(common.SEED)

[123, 151010689]

In [3]:
net = dqn_model.DQN(env.observation_space.shape, env.action_space.n).to(device)

tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params.epsilon_start)
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(net, selector, device = device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma, steps_count=4)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr=params.learning_rate)

In [4]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = common.calc_loss_dqn(
        batch, net, tgt_net.target_model,
        gamma=params.gamma*4, device=device)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()
    return {
        "loss": loss_v.item(),
        "epsilon": selector.epsilon,
    }

engine = Engine(process_batch)
common.setup_ignite(engine, params, exp_source, f"{NAME}={4}")
engine.run(common.batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-21, steps=843, speed=0.0 f/s, elapsed=0:01:06
Episode 2: reward=-20, steps=1054, speed=0.0 f/s, elapsed=0:01:06
Episode 3: reward=-21, steps=880, speed=0.0 f/s, elapsed=0:01:06
Episode 4: reward=-21, steps=822, speed=0.0 f/s, elapsed=0:01:06
Episode 5: reward=-20, steps=914, speed=0.0 f/s, elapsed=0:01:06
Episode 6: reward=-21, steps=911, speed=0.0 f/s, elapsed=0:01:06
Episode 7: reward=-21, steps=820, speed=0.0 f/s, elapsed=0:01:06
Episode 8: reward=-20, steps=865, speed=0.0 f/s, elapsed=0:01:06
Episode 9: reward=-20, steps=840, speed=0.0 f/s, elapsed=0:01:06
Episode 10: reward=-20, steps=1072, speed=0.0 f/s, elapsed=0:01:06
Episode 11: reward=-21, steps=888, speed=0.0 f/s, elapsed=0:01:06
Episode 12: reward=-20, steps=921, speed=62.6 f/s, elapsed=0:01:19
Episode 13: reward=-20, steps=977, speed=62.7 f/s, elapsed=0:01:33
Episode 14: reward=-20, steps=901, speed=62.7 f/s, elapsed=0:01:48
Episode 15: reward=-20, steps=1047, speed=62.8 f/s, elapsed=0:02:03
Episode 16: 

Engine run is terminating due to exception: .


KeyboardInterrupt: 